In [ ]:
%matplotlib notebook

In [1]:
import sys, time
sys.path.append('/home/pi/Desktop/ADR_NX/Utils')
sys.path.append('/home/pi/Desktop/ADR_NX/Mainframe')
import adr_utils, SIM900_controller, SIM921, SIM922, SIM960, adr_PID
import numpy as np
import matplotlib.pyplot as plt
from timeit import default_timer as timer
mfc = SIM900_controller.SIM900()
mfc.init_and_open_connection()
mfc.start_input_loop()
mfc.start_output_loop()
s921 = SIM921.SIM921(mfc)
s922 = SIM922.SIM922(mfc)
s960 = SIM960.SIM960(mfc)
mfc.get_serial_object()

Input loop startedInput read thread is up!

 Output loop started


Serial<id=0x71a9ea90, open=True>(port='/dev/ADR_sim900', baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=0, xonxoff=False, rtscts=False, dsrdtr=False)

In [ ]:
mfc.set_RPER([s921.MODNUM, s922.MODNUM, s960.MODNUM])

In [ ]:
mfc.reset_serial_module(1)

In [ ]:
print s921.get_single_value()
print s960.is_inmanualmode()

In [ ]:
print s921.get_single_value()

In [ ]:
s921.set_data_period(2000)

In [3]:
s921.start_data_stream()
s921.newdataevent.wait(20)
print "NEW DATA"
time.sleep(5)
s921.stop_data_stream()

OQADD (t:0): |SNDT 1,"TVAL? 0\n"\n|
NEW DATA
OQSEND (t:0): |SNDT 1,"TVAL? 0\n"\n|
Read line: 'MSG 1,#215+2.955420E+00\r\n'
GOT MSG -> S1:+2.955420E+00
Got temperature data: +2.955420E+00|21:35:27.270150
Read line: 'MSG 1,#215+2.963549E+00\r\n'
GOT MSG -> S1:+2.963549E+00
Got temperature data: +2.963549E+00|21:35:29.261556
Read line: 'MSG 1,#215+2.959560E+00\r\n'
GOT MSG -> S1:+2.959560E+00
Got temperature data: +2.959560E+00|21:35:31.257737
OQADD (t:0): |SNDT 1,"SOUT \n"\n|
OQSEND (t:0): |SNDT 1,"SOUT \n"\n|


In [ ]:
list(s921.templist)

In [ ]:
s921.stop_data_stream()

In [ ]:
plt.figure(1)
plt.plot(list(s921.templist),list(s921.templist))

In [ ]:
mfc.get_IDN()

In [ ]:
mfc.get_serial_object().write("*IDN?\n")

In [ ]:
mfc.get_serial_object().read(1024)

In [ ]:
mfc.get_serial_object().close()

In [ ]:
mfc.get_serial_object().write("SNDT 1,\"SOUT\n\"\n")

In [ ]:
print s921.get_last_time()
print s921.get_last_temperature()

In [ ]:
mfc.stop_input_loop()
mfc.stop_output_loop()

In [ ]:
target = 200

In [ ]:
P = 1.8
I = 0.0
D = 0.0

pid = ADR_PID(P,I,D,1)
pid.SetPoint=0.0
pid.setSlewLimit(10.0)

END = 80
feedback = 0
debug = 1

feedback_list = []
time_list = []
setpoint_list = []

In [ ]:
for i in range(1, END):
    if i>9:
        pid.SetPoint = 2

    pid.update(feedback)
    output = pid.output
    if pid.SetPoint > 0:
        feedback += (output - (1/i))
    time.sleep(0.02)

    feedback_list.append(feedback)
    setpoint_list.append(pid.SetPoint)
    time_list.append(i)

#time_sm = np.array(time_list)
#time_smooth = np.linspace(time_sm.min(), time_sm.max(), 300)
#feedback_smooth = spline(time_list, feedback_list, time_smooth)

In [ ]:
plt.figure(1)
plt.plot(time_list, feedback_list)
plt.plot(time_list, setpoint_list)
plt.xlim((0, END))
#plt.ylim((min(feedback_list)-0.5, max(feedback_list)+0.5))
#plt.ylim((0,3))
plt.xlabel('time (s)')
plt.ylabel('PID (PV)')
plt.title('TEST PID')

#plt.ylim((1-0.5, 1+0.5))
plt.ylim((0,3))

plt.grid(True)
plt.show()

In [ ]:
plt.figure(2)
plt.plot(time_list, [(feedback_list[i] - setpoint_list[i]) for  i in range(0,len(feedback_list))])
plt.xlim((0, END))
#plt.ylim((min(feedback_list)-0.5, max(feedback_list)+0.5))
#plt.ylim((0,3))
plt.xlabel('time (s)')
plt.ylabel('PID (PV)')
plt.title('TEST PID')

#plt.ylim((1-0.5, 1+0.5))
plt.ylim((0,3))

plt.grid(True)
plt.show()